In [1]:
# Imports
import struct
import traceback
import math
import os
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import open3d as o3d
from scipy.spatial import Delaunay
from functools import reduce
import copy
# %matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Constants
# Ethernet ----------------------------------------------------------------
SENSOR_FRONT_IP = "192.168.1.10"
SENSOR_RIGHT_IP = "192.168.1.11"
SENSOR_LEFT_IP = "192.168.1.12"
SENSOR_TOP_IP = "192.168.1.13"

# Scans -------------------------------------------------------------------
SCANS_DIRECTORY = r'C:\Users\victo\OneDrive\Desktop\Studies\2024.1\Projeto Integrador\Projeto\DAS5104_Projeto_LiDAR\pointcloud'
SCAN_DIRECTION = "ccw"

# Sensors -----------------------------------------------------------------
SENSOR_TOP_HEIGHT = 2400

# [sensor_x_offset, sensor_y_offset, sensor_z_offset]
SENSOR_RIGHT_TRANSLATION = (1070, 1130, 0)
SENSOR_LEFT_TRANSLATION = (1105, -1130, 0)

# Euler angles [x, y, z]
SENSOR_RIGHT_ROTATION = (0, 0, 0)
SENSOR_LEFT_ROTATION = (0, 0, 0)

BOUNDARIES_PROFILE_X_MIN = 100
BOUNDARIES_PROFILE_X_MAX = 2385
BOUNDARIES_PROFILE_Y_MIN = -1000
BOUNDARIES_PROFILE_Y_MAX = 1000
BOUNDARIES_PROFILE_Z_MIN = -2750
BOUNDARIES_PROFILE_Z_MAX = -500

BOUNDARIES_ZAXIS_X_MIN = -2300
BOUNDARIES_ZAXIS_X_MAX = -400
BOUNDARIES_ZAXIS_Y_MIN = -7000
BOUNDARIES_ZAXIS_Y_MAX = -100

# Output -----------------------------------------------------------------
OUTPUT_DIRECTORY = r'outputs'

TRUCK_VELOCITY = 10

In [3]:
def ntp64_to_seconds(integer):
    # Upper 32 bits for seconds
    seconds = integer >> 32

    # Lower 32 bits for fractional seconds
    fractional_seconds = integer & 0xFFFFFFFF
    fractional_seconds = fractional_seconds / 0x100000000

    return round(seconds + fractional_seconds, 3)

In [4]:
def polar_to_xy(distances: list, first_angle: int, angular_increment: int):
    first_angle /= 10000
    angular_increment /= 10000

    xy = list()

    for i, distance in enumerate(distances):
        # Invalid measurements return 0xFFFFFFFF
        if distance == 4_294_967_295:
            continue

        angle = (first_angle + i * angular_increment) * math.pi / 180.0

        x = round(distance * math.cos(angle))
        y = round(distance * math.sin(angle))

        xy.append((x, y))

    return xy

In [5]:
def process_binary_file(file_path: str):
    file = open(file_path, "rb")
    data = file.read()
    file.close()

    scans = dict()
    magic_byte = struct.pack("H", 0xa25c)

    for packet in data.split(magic_byte):

        if len(packet) <= 10:
            continue

        try:
            packet_size = struct.unpack("I", packet[2:6])[0] - len(magic_byte)
            header_size = struct.unpack("H", packet[6:8])[0] - len(magic_byte)
            scan_number = struct.unpack("H", packet[8:10])[0]
            timestamp_raw = struct.unpack("Q", packet[12:20])[0]
            first_angle = struct.unpack("i", packet[42:46])[0]
            angular_increment = struct.unpack("i", packet[46:50])[0]
        except Exception as e:
            print(f"Package from IP {os.path.basename(file_path).split('.bin')[0]} corrupted...")
            print(e)
            continue

        if len(packet) != packet_size:
            print(f"Package from IP {os.path.basename(file_path).split('.bin')[0]} corrupted")
            print('Expected packet size:', packet_size)
            print('Received packet size:', len(packet))
            continue

        if scan_number not in scans:
            scans[scan_number] = dict()
            scans[scan_number]["xy"] = list()
            scans[scan_number]["timestamp"] = ntp64_to_seconds(timestamp_raw)

        payload = packet[header_size:]  # list[uint32] - 4byte
        distances = struct.unpack(f"{len(payload) // 4}I", payload[:len(payload) // 4 * 4])

        scans[scan_number]["xy"].extend(polar_to_xy(distances, first_angle, angular_increment))

    return scans

In [6]:
def calculate_z_axis(scans_front: dict, x_min: int, x_max: int, y_min: int, y_max: int, velocity: int):
    z_axis = {}
    xyz_front = []

    for i, scan_key in enumerate(sorted(scans_front.keys())):
        z_axis[i] = y_min

        for xy in scans_front[scan_key]["xy"]:
            x = xy[0]
            y = xy[1]
            z = i * velocity

            if x <= x_min or x >= x_max or y <= y_min or y >= y_max:
                continue

            if y > z_axis[i]:
                z_axis[i] = y

            xyz_front.append((x, y, z))

    return z_axis, xyz_front

In [7]:
def reconstruct_z_axis(scans: dict, z_axis: dict) -> list[tuple[int, int, int]]:
    xyz = list()

    for key in zip(sorted(scans.keys()), sorted(z_axis.keys())):
        for xy in scans[key[0]]["xy"]:
            x = xy[0]
            y = xy[1]
            z = z_axis[key[1]]

            xyz.append((x, y, z))

    return xyz

In [8]:
def transform(points, rotation: tuple[int, int, int], translation: tuple[int, int, int]):
    try:
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)
    
        if rotation != (0, 0, 0):
            rotation_matrix = pcd.get_rotation_matrix_from_xyz(rotation)
            pcd.rotate(rotation_matrix, center=(0, 0, 0))

        if translation != (0, 0, 0):
            pcd.translate(translation)
    except Exception as e:
        print(f'Error in point {points}: {traceback.format_exc()}')

    return np.asarray(pcd.points)

In [9]:
def remove_boundaries(points, x_min: int, x_max: int, y_min: int, y_max: int):
  return [p for p in points if not (p[0] <= x_min or p[0] >= x_max or p[1] <= y_min or p[1] >= y_max)]

In [10]:
def plot_point_cloud(arr, title=''):
  df=pd.DataFrame(data=arr, columns=['x', 'y', 'z'])
  fig = px.scatter_3d(df, x='x', y='y', z='z', color='y', title=title, color_continuous_scale=px.colors.sequential.Cividis_r)
  fig.update_traces(marker_size=2)
  fig.show()

In [11]:
def filter_point_cloud(points, voxel_size, nb_neighbors, std_ratio, nb_points, radius):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(points)

  # xyz_1 = pcd.voxel_down_sample(voxel_size)
  # xyz_1 = pcd.uniform_down_sample(every_k_points=2)
  xyz_2, _ = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
  xyz_3, _ = xyz_2.remove_radius_outlier(nb_points=nb_points, radius=radius)

  return np.asarray(xyz_3.points)

In [12]:
def get_point_cloud(obj, velocity):
  scans_front = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_FRONT_IP}.bin'))
  scans_right = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_RIGHT_IP}.bin'))
  scans_left = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_LEFT_IP}.bin'))
  scans_top = process_binary_file(os.path.join(SCANS_DIRECTORY, obj, f'{SENSOR_TOP_IP}.bin'))
  
  z_axis, _ = calculate_z_axis(
    scans_front,
    BOUNDARIES_ZAXIS_X_MIN,
    BOUNDARIES_ZAXIS_X_MAX,
    BOUNDARIES_ZAXIS_Y_MIN,
    BOUNDARIES_ZAXIS_Y_MAX,
    velocity
  )
  
  xyz_right = reconstruct_z_axis(scans_right, z_axis)
  # plot_point_cloud(xyz_right, 'Scans do sensor right com reconstrução do eixo Z')
  xyz_left = reconstruct_z_axis(scans_left, z_axis)
  xyz_top = reconstruct_z_axis(scans_top, z_axis)
  
  xyz_right = transform(xyz_right, SENSOR_RIGHT_ROTATION, SENSOR_RIGHT_TRANSLATION)
  # plot_point_cloud(xyz_right, 'Scans do sensor right após translação')
  xyz_left = transform(xyz_left, SENSOR_LEFT_ROTATION, SENSOR_LEFT_TRANSLATION)
  
  xyz_right = remove_boundaries(
    xyz_right,
    BOUNDARIES_PROFILE_X_MIN,
    BOUNDARIES_PROFILE_X_MAX,
    BOUNDARIES_PROFILE_Y_MIN,
    BOUNDARIES_PROFILE_Y_MAX,
  )
  
  # plot_point_cloud(xyz_right, 'Scans do sensor right sem dados fora dos limites')

  xyz_left = remove_boundaries(
    xyz_left,
    BOUNDARIES_PROFILE_X_MIN,
    BOUNDARIES_PROFILE_X_MAX,
    BOUNDARIES_PROFILE_Y_MIN,
    BOUNDARIES_PROFILE_Y_MAX,
  )

  xyz_top = remove_boundaries(
    xyz_top,
    BOUNDARIES_PROFILE_X_MIN,
    BOUNDARIES_PROFILE_X_MAX,
    BOUNDARIES_PROFILE_Y_MIN,
    BOUNDARIES_PROFILE_Y_MAX,
  )

  xyz_right = filter_point_cloud(xyz_right, 15, 40, 0.1, 25, 50)
  xyz_left = filter_point_cloud(xyz_left, 15, 40, 0.1, 25, 50)
  xyz_top = filter_point_cloud(xyz_top, 15, 60, 0.06, 25, 120)
  
  xyz = list()
  xyz.extend(xyz_right)
  xyz.extend(xyz_left)
  xyz.extend(xyz_top)

  xyz = transform(xyz, (0, 0, -math.pi/2), (0, SENSOR_TOP_HEIGHT, 0))
  # plot_point_cloud(xyz, 'Point cloud reconstruída')

  return xyz

In [13]:
def convex_hull(xyz):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(xyz)
  pcd.estimate_normals()

  hull, _ = pcd.compute_convex_hull()
  hull.orient_triangles()

  hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
  hull_ls.paint_uniform_color((1, 0, 0))

  volume = hull.get_volume()

  # o3d.visualization.draw_geometries([pcd, hull])

  return volume

In [14]:
def alpha_shapes(xyz):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(xyz)
  pcd.estimate_normals()

  alpha = 25

  mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
  mesh.orient_triangles()

  volume = mesh.get_volume()
  
  # o3d.visualization.draw_geometries([pcd, mesh])

  return volume

In [15]:
def _volume_under_triangle(triangle):
    p1, p2, p3 = triangle
    x1, z1, y1 = p1
    x2, z2, y2 = p2
    x3, z3, y3 = p3

    return abs((z1+z2+z3)*(x1*y2-x2*y1+x2*y3-x3*y2+x3*y1-x1*y3)/6)

def _get_triangles_vertices(triangles, vertices):
    triangles_vertices = []

    for triangle in triangles:
        new_triangles_vertices = [vertices[triangle[0]], vertices[triangle[1]], vertices[triangle[2]]]
        triangles_vertices.append(new_triangles_vertices)

    return np.array(triangles_vertices)

def delaunay(xyz):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    xyz = np.asarray(pcd.points)
    xy_catalog = []

    for point in xyz:
        xy_catalog.append([point[0], point[2]])

    tri = Delaunay(np.array(xy_catalog))

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(xyz)
    mesh.triangles = o3d.utility.Vector3iVector(tri.simplices)
    
    # o3d.visualization.draw_geometries([pcd, mesh])

    volume = reduce(lambda a, b:  a + _volume_under_triangle(b),
                    _get_triangles_vertices(mesh.triangles, mesh.vertices), 0)

    return volume

In [16]:
def ball_pivoting(xyz):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(xyz)
  pcd.estimate_normals()

  radii = [50, 25, 30]
  mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd, o3d.utility.DoubleVector(radii))

  volume = mesh.get_volume()
  return volume


In [17]:
def estimate_volume(xyz):
  try:
    print(f'Convex hull: {convex_hull(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método do convex hull:', e)
  try:
    print(f'Alpha shapes: {alpha_shapes(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método das alpha shapes:', e)
  try:
    print(f'Ball pivoting: {ball_pivoting(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método de ball pivoting:', e)
  try:
    print(f'Triangulação de Delaunay: {delaunay(xyz)} mm³')
  except Exception as e:
    print('O volume não pode ser estimado através do método da triangulação de delaunay:', e)

In [18]:
def get_bucket_bottom_height(point_cloud):
  n_points = {}
  for p in point_cloud:
    height = round(p[1])
    if height in n_points:
      n_points[height] += 1
    else:
      n_points[height] = 1

  return max(n_points, key=n_points.get)


In [19]:
def get_min_coordinates(point_cloud):
  return min(point_cloud, key=lambda p: p[0])[0], min(point_cloud, key=lambda p: p[1])[1], min(point_cloud, key=lambda p: p[2])[2]

In [20]:
def get_max_coordinates(point_cloud):
  return max(point_cloud, key=lambda p: p[0])[0], max(point_cloud, key=lambda p: p[1])[1], max(point_cloud, key=lambda p: p[2])[2]

In [21]:
def preprocess_point_cloud(pcd, voxel_size, max_nn_normals, max_nn_fpfh):
    # print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    # print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normals))

    radius_feature = voxel_size * 5
    # print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))
    return pcd_down, pcd_fpfh

In [22]:
def ransac_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, max_iteration, confidence):
  distance_threshold = voxel_size * 1.5
  print(":: RANSAC registration on downsampled point clouds.")
  print("   Since the downsampling voxel size is %.3f," % voxel_size)
  print("   we use a liberal distance threshold %.3f." % distance_threshold)
  result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
    source_down, target_down, source_fpfh, target_fpfh, True,
    distance_threshold,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
    3, [
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
            0.1),
        o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
            distance_threshold)
    ], o3d.pipelines.registration.RANSACConvergenceCriteria(max_iteration, confidence))
  
  return result

In [23]:
def icp_registration(source, target, trans_init, voxel_size, method='point_to_point', epsilon=1e-4, max_iteration=30):
  distance_threshold = voxel_size * 0.4
  target.estimate_normals()

  convergence_criteria = o3d.pipelines.registration.ICPConvergenceCriteria(
    relative_fitness=1e-6,
    relative_rmse=1e-6,
    max_iteration=max_iteration)
  if method == 'generalized':
    print(":: Generalized ICP registration is applied on original point")
    estimation_method = o3d.pipelines.registration.TransformationEstimationForGeneralizedICP(epsilon)
    result = o3d.pipelines.registration.registration_generalized_icp(
      source,
      target,
      distance_threshold,
      trans_init,
      estimation_method,
      convergence_criteria)
    return result
  if method == 'point_to_point':
    print(":: Point-to-point ICP registration is applied on original point")
    estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint()
  elif method == 'point_to_plane':
    print(":: Point-to-plane ICP registration is applied on original point")
    estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPlane()
  else:
    raise ValueError("Informed method not allowed. Available methods: 'point_to_point', 'point_to_plane', 'generalized'.")
  
  print("   clouds to refine the alignment. This time we use a strict")
  print("   distance threshold %.3f." % distance_threshold)

  result = o3d.pipelines.registration.registration_icp(
    source, target, distance_threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    convergence_criteria
  )
  
  return result

In [24]:
def align_truck_bucket_and_load(load, bucket, voxel_size, max_iteration, confidence, max_nn_normals, max_nn_fpfh):
  source_down, source_fpfh = preprocess_point_cloud(load, voxel_size, max_nn_normals, max_nn_fpfh)
  target_down, target_fpfh = preprocess_point_cloud(bucket, voxel_size, max_nn_normals, max_nn_fpfh)
  
  result_ransac = ransac_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, max_iteration, confidence)
  
  return result_ransac.transformation, result_ransac.fitness, result_ransac.inlier_rmse

  result_icp = icp_registration(load, bucket, result_ransac.transformation, voxel_size)
  print('ICP result:', result_icp)
  print('ICP resulting transformation:', result_icp.transformation)  
  
  aligned = copy.deepcopy(load)
  aligned.transform(result_ransac.transformation)

  return aligned

In [25]:
bucket_dir = 'caixa_vazia'
bucket_points = get_point_cloud(bucket_dir, 5)
bucket_bottom_y = get_bucket_bottom_height(bucket_points)
bucket_points = [p for p in bucket_points if -2750 < p[2] < -500 and p[1] >= bucket_bottom_y - 10]
# bucket_points = [p for p in bucket_points if -2750 < p[2] < -500]

load_dir = 'caixa_brita_pico'
load_points = get_point_cloud(load_dir, 5)
load_points = [p for p in load_points if -2750 < p[2] < -500 and p[1] >= bucket_bottom_y - 10]
# load_points = [p for p in load_points if -2750 < p[2] < -500]

source = o3d.geometry.PointCloud()
source.points = o3d.utility.Vector3dVector(load_points)
target = o3d.geometry.PointCloud()
target.points = o3d.utility.Vector3dVector(bucket_points)

x1 = [p[0] for p in target.points]
y1 = [p[1] for p in target.points]
z1 = [p[2] for p in target.points]

x2 = [p[0] for p in source.points]
y2 = [p[1] for p in source.points]
z2 = [p[2] for p in source.points]

trace1 = go.Scatter3d(
    x=x1, y=y1, z=z1,
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
    ),
    name='Empty truck'
)

trace2 = go.Scatter3d(
    x=x2, y=y2, z=z2,
    mode='markers',
    marker=dict(
        size=3,
        color='red',
    ),
    name='Truck with load'
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)

# Update the layout
fig.update_layout(
    legend=dict(
        x=0.8,  # Adjust x-coordinate
        y=0.8,  # Adjust y-coordinate
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Show the figure
fig.show()

Package from IP 192.168.1.12 corrupted...
unpack requires a buffer of 8 bytes
Package from IP 192.168.1.12 corrupted
Expected packet size: -2
Received packet size: 1385
Package from IP 192.168.1.12 corrupted
Expected packet size: 1402
Received packet size: 60
Package from IP 192.168.1.12 corrupted
Expected packet size: -2
Received packet size: 1340


In [26]:

voxel_size = 30
max_nn_normals = 300
max_nn_fpfh = 1000
confidence = 1.0
max_iteration_ransac = 100000

source_down, source_fpfh = preprocess_point_cloud(source, voxel_size, max_nn_normals, max_nn_fpfh)
target_down, target_fpfh = preprocess_point_cloud(target, voxel_size, max_nn_normals, max_nn_fpfh)
  
result_ransac = ransac_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size, max_iteration_ransac, confidence)
print('RANSAC result:', result_ransac)
print('RANSAC transformation:', result_ransac.transformation)

epsilon = 1e-6
max_iteration_icp = 2000
# result_icp = g_icp_registration(source, target, result_ransac.transformation, voxel_size, epsilon, max_iteration_icp)
result_icp = icp_registration(source, target, result_ransac.transformation, voxel_size, 'generalized', epsilon, max_iteration_icp)


print('ICP result:', result_icp)
print('ICP transformation:', result_icp.transformation)

icp_aligned = copy.deepcopy(source)
icp_aligned.transform(result_icp.transformation)

ransac_aligned = copy.deepcopy(source)
ransac_aligned.transform(result_ransac.transformation)

x1 = [p[0] for p in target.points]
y1 = [p[1] for p in target.points]
z1 = [p[2] for p in target.points]

x2 = [p[0] for p in source.points]
y2 = [p[1] for p in source.points]
z2 = [p[2] for p in source.points]

x3 = [p[0] for p in ransac_aligned.points]
y3 = [p[1] for p in ransac_aligned.points]
z3 = [p[2] for p in ransac_aligned.points]

x4 = [p[0] for p in icp_aligned.points]
y4 = [p[1] for p in icp_aligned.points]
z4 = [p[2] for p in icp_aligned.points]

trace1 = go.Scatter3d(
    x=x1, y=y1, z=z1,
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
    ),
    name='Empty truck'
)

trace2 = go.Scatter3d(
    x=x2, y=y2, z=z2,
    mode='markers',
    marker=dict(
        size=3,
        color='red',
    ),
    name='Truck with load (not aligned)'
)

trace3 = go.Scatter3d(
    x=x3, y=y3, z=z3,
    mode='markers',
    marker=dict(
        size=3,
        color='green',
    ),
    name='Truck with load (RANSAC aligned)'
)

trace4 = go.Scatter3d(
    x=x4, y=y4, z=z4,
    mode='markers',
    marker=dict(
        size=3,
        color='purple',
    ),
    name='Truck with load (G-ICP aligned)'
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.add_trace(trace4)

# Update the layout

fig.update_layout(
    legend=dict(
        x=0.8,  # Adjust x-coordinate
        y=0.8,  # Adjust y-coordinate
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Show the figure
fig.show()

:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 30.000,
   we use a liberal distance threshold 45.000.
RANSAC result: RegistrationResult with fitness=7.601093e-01, inlier_rmse=2.106252e+01, and correspondence_set size of 2782
Access transformation to get result.
RANSAC transformation: [[ 9.99737930e-01 -1.27864915e-02  1.89888786e-02 -2.68063289e+00]
 [ 1.36668971e-02  9.98802227e-01 -4.69821957e-02 -1.75569074e+02]
 [-1.83653968e-02  4.72294021e-02  9.98715223e-01  1.30427169e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
:: Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. This time we use a strict
   distance threshold 12.000.
ICP result: RegistrationResult with fitness=4.703669e-01, inlier_rmse=8.334302e+00, and correspondence_set size of 3500
Access transformation to get result.
ICP transformation: [[ 9.99826913e-01  1.00562106e-02  1.56530049e-02 -3.54581306e+01]
 [

In [ ]:
kd_tree = o3d.geometry.KDTreeFlann(target)
inner_load_points = []
threshold_distance = 20
nb_neighbors = 40
std_ratio = 1
nb_points = 10
radius = 100

for point in ransac_aligned.points:
    [_, idx, _] = kd_tree.search_knn_vector_3d(point, 1)
    closest_point = target.points[idx[0]]
    if np.linalg.norm(np.array(point) - np.array(closest_point)) > threshold_distance:
        inner_load_points.append(point)

print(f"Size before:", len(ransac_aligned.points))
print(f"Size after:", len(inner_load_points))

inner_load_pc = o3d.geometry.PointCloud()
inner_load_pc.points = o3d.utility.Vector3dVector(inner_load_points)

filtered, _ = inner_load_pc.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)
filtered, _ = filtered.remove_radius_outlier(nb_points=nb_points, radius=radius)

x1 = [p[0] for p in ransac_aligned.points]
y1 = [p[1] for p in ransac_aligned.points]
z1 = [p[2] for p in ransac_aligned.points]

x2 = [p[0] for p in inner_load_pc.points]
y2 = [p[1] for p in inner_load_pc.points]
z2 = [p[2] for p in inner_load_pc.points]

x3 = [p[0] for p in filtered.points]
y3 = [p[1] for p in filtered.points]
z3 = [p[2] for p in filtered.points]

x4 = [p[0] for p in target.points]
y4 = [p[1] for p in target.points]
z4 = [p[2] for p in target.points]

trace1 = go.Scatter3d(
    x=x1, y=y1, z=z1,
    mode='markers',
    marker=dict(
        size=3,
        color='blue',
    ),
    name='Original'
)

trace2 = go.Scatter3d(
    x=x2, y=y2, z=z2,
    mode='markers',
    marker=dict(
        size=3,
        color='red',
    ),
    name='Bucket removed'
)

trace3 = go.Scatter3d(
    x=x3, y=y3, z=z3,
    mode='markers',
    marker=dict(
        size=3,
        color='pink',
    ),
    name='Filtered bucket removed'
)

trace4 = go.Scatter3d(
    x=x4, y=y4, z=z4,
    mode='markers',
    marker=dict(
        size=3,
        color='green',
    ),
    name='Truck bucket'
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.add_trace(trace3)
fig.add_trace(trace4)

# Update the layout
fig.update_layout(
     legend=dict(
        x=0.8,  # Adjust x-coordinate
        y=0.8,  # Adjust y-coordinate
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z')
    )
)

# Show the figure
fig.show()

In [ ]:
obj = 'caixa_brita_uniforme_2024-05-06_11h33min44s'

uniforme = get_point_cloud(obj, 5)
uniforme = [p for p in uniforme if p[2] > -6000 and p[2] < -1000]
plot_point_cloud(uniforme)

In [ ]:
obj = 'caixa_brita_vao_2024-05-06_11h38min15s'

vao = get_point_cloud(obj, 5)
vao = [p for p in vao if p[2] < -1000]
plot_point_cloud(vao)

In [ ]:
obj = 'caixa_vazia_2024-05-06_11h22min41s'

vazio = get_point_cloud(obj, 5)
vazio = [p for p in vazio if p[2] > -6000 and p[2] < -1000]

obj = 'caixa_brita_pico_2024-05-06_11h41min34s'

pico = get_point_cloud(obj, 5)
pico = [p for p in pico if p[2] < -1000]

pico_translate = [[p[0] - 50, p[1], p[2]] for p in pico]
points = [p for p in vazio + pico_translate if p[1] > 400]

plot_point_cloud(points)

In [ ]:
pcd = o3d.geometry.PointCloud()
# pcd = copy.deepcopy(filtered)
pcd = copy.deepcopy(target)
# pcd.points = o3d.utility.Vector3dVector(list(filtered.points) + list(target.points))
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(100)

# o3d.visualization.draw_geometries([pcd], point_show_normal=True)

In [ ]:
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd)
bbox = pcd.get_axis_aligned_bounding_box()
mesh = mesh.crop(bbox)

# Refine the mesh
mesh = mesh.filter_smooth_simple(number_of_iterations=1)
mesh.paint_uniform_color([0.7, 0.7, 0.7])

# Save and visualize the mesh
o3d.io.write_triangle_mesh("truck.ply", mesh)
# o3d.visualization.draw_geometries([pcd], point_show_normal=True)
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True, mesh_show_wireframe=True)